# Denison CS181/DA210 SW Lab #10 - Step 1

Before you complete the lab checkpoints, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells** (in the menubar, select Kernel$\rightarrow$Restart And Run All).

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

#### Import Python modules and load "SQL Magic"

In [1]:
import pandas as pd
import os
import os.path
import json
import sys
import importlib

module_dir = "../../modules"
module_path = os.path.abspath(module_dir)
if not module_path in sys.path:
    sys.path.append(module_path)

%load_ext sql

In [2]:
def getsqlite_creds(dirname=".",filename="creds.json"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D["sqlite1"]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [3]:
scheme, dbdir, database = getsqlite_creds()
template = '{}:///{}/{}.db'
cstring = template.format(scheme, dbdir, database)
print("Connection string:", cstring)

Connection string: sqlite:///../../dbfiles/book.db


#### Establish Connection from Client to Server

In [4]:
%sql $cstring

---

# Part A: SQL Grammar (Recap)

So far, we have seen the following SQL grammar:

```
    SELECT field-spec
    FROM table-spec
    [order-clause]
    [limit-clause]
```

For example, we may want to project the `pop` and `gdp` columns from `indicators`, but only for the country/year combinations with the 5 highest life expectancies:

```SQL
    SELECT pop AS Population, gdp AS GDP
    FROM indicators
    ORDER BY life DESC
    LIMIT 5
```

We can execute this command using a single-line SQL statement, as follows:

In [5]:
%sql SELECT pop AS Population, gdp AS GDP FROM indicators ORDER BY life DESC LIMIT 5

 * sqlite:///../../dbfiles/book.db
Done.


Population,GDP
0.03,1.8
7.39,341.65
7.29,309.38
7.34,320.86
126.79,4859.95


That's pretty hard to read, so we can instead build our SQL query as a multi-line Python string, and pass that string to the "SQL magic":

In [6]:
query = """
SELECT pop AS Population, gdp AS GDP
FROM indicators
ORDER BY life DESC
LIMIT 5
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


Population,GDP
0.03,1.8
7.39,341.65
7.29,309.38
7.34,320.86
126.79,4859.95


---

# Part B: SQL Single-Table Selection and Row Filtering

We can expand on the multi-column projection queries we've looked at so far to include filtering based on the values in a given field (column).

As a simple example, we can select only the distinct values of a given field.  To do so, we'll add an optional `DISTINCT` keyword to our grammar (recall that `[]` indicate that the contents are optional):
```
    SELECT [DISTINCT] field-spec
    FROM table-spec
    [order-clause]
    [limit-clause]
```

For example, we may want the unique country codes from the `indicators` table in the `book` database:

In [7]:
query = """
SELECT DISTINCT code
FROM indicators
"""

# Get the results as a DataFrame so we don't have to view the whole thing (it's long)
resultset = %sql $query
resultdf = resultset.DataFrame()
resultdf

 * sqlite:///../../dbfiles/book.db
Done.


,code
0,ABW
1,AFG
2,AGO
3,ALB
4,AND
...,...
213,XKX
214,YEM
215,ZAF
216,ZMB


Alternatively, we could filter based on a filter condition.  For this, we'll add one new clause to our grammar:
```
    SELECT [DISTINCT] field-spec
    FROM table-spec
    [WHERE [NOT] filter-cond]
    [order-clause]
    [limit-clause]
```

We will look at several types of filter conditions:
1. Binary operations and Booleans
2. Range of values
3. In a set
4. Like (matching) a pattern
5. Null/not null

### 1. Binary operations/Booleans

Filter conditions using binary operations and Booleans have the form `expr op expr`, using the following possible choices for `op`: `=`, `<>`, `<`, `>`, `<=`, `>=`, `NOT`, `AND`, `OR`.

For example:

In [8]:
query = """
SELECT code, pop
FROM indicators
WHERE gdp > 5000 AND year = 2012
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


code,pop
CHN,1350.7
JPN,127.63
USA,313.87


In [9]:
query = """
SELECT name, count
FROM topnames
WHERE year >= 2015 AND year < 2018 AND sex <> 'Female'
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


name,count
Noah,19635
Noah,19117
Liam,18798


### 2. Range of values

The last example above could be simplified by instead specifying that we want the year between `2015` and `2017`.  We can do that using a range clause:

```
    range-clause |= expr [NOT] BETWEEN low AND high
```

For example:

In [10]:
query = """
SELECT name, count
FROM topnames
WHERE (year BETWEEN 2015 AND 2017) AND sex <> 'Female'
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


name,count
Noah,19635
Noah,19117
Liam,18798


### 3. In a set

If we instead want to find the records for which a given value is one of a set, we can use an `IN` clause:

```
    in-clause |= expr [NOT] IN (set-members)
```

For example:

In [11]:
query = """
SELECT code, pop
FROM indicators
WHERE year = 2017 AND code IN ('VNM', 'KOR')
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


code,pop
KOR,51.47
VNM,94.6


### 4. Like (matching) a pattern

If we want the records for which a given value matches part of a string pattern, we can use a `LIKE` clause:

```
    like-clause |= expr [NOT] LIKE pattern
```

With a `LIKE` clause, we can use either `%` to match multiple characters or `_` to match a single character.

For example:

In [12]:
query = """
SELECT code, country, region
FROM countries
WHERE country LIKE '%Republic' OR code LIKE '_ZA'
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


code,country,region
CAF,Central African Republic,Sub-Saharan Africa
CZE,Czech Republic,Europe & Central Asia
DOM,Dominican Republic,Latin America & Caribbean
DZA,Algeria,Middle East & North Africa
KGZ,Kyrgyz Republic,Europe & Central Asia
SVK,Slovak Republic,Europe & Central Asia
SYR,Syrian Arab Republic,Middle East & North Africa
TZA,Tanzania,Sub-Saharan Africa


### 5. Null/not null

If we want the records for which a given value is or is not missing, we can use a `NULL` clause:

```
    null-clause |= expr IS [NOT] NULL
```

For example:

In [13]:
query = """
SELECT code, country, land
FROM countries
WHERE land IS NULL
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


code,country,land
CUW,Curacao,None
MAF,St. Martin (French part),None
MCO,Monaco,None
SDN,Sudan,None
SSD,South Sudan,None
SXM,Sint Maarten (Dutch part),None
XKX,Kosovo,None


---

## Part C: Try it yourself

In the following cells, your only action is to remove the `# YOUR CODE HERE` and `raise NotImplementedError()` lines, and then put a valid SQL statement as the **value** of string variable `query`.

In each case, when you execute the cell, the query will be sent to the database management system, a result obtained, and the result converted into a `pandas` data frame, whose prefix is shown.  (This allows for testing the results as well as displaying them in your Jupyter Notebook.)

_Note: you may want to view the `book` database in SQLiteStudio to get an idea of what the field names are as you work through these exercises.  Alternatively, start by adding a few code cells to perform a `SELECT * FROM tablename` query for each table name._

**Q1:** Using the table `indicators`, write a query to find all unique years that appear in the table.

In [14]:
query = """
SELECT DISTINCT year
FROM indicators
"""
# YOUR CODE HERE
# raise NotImplementedError()

resultset1 = %sql $query
resultdf1 = resultset1.DataFrame()
resultdf1.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year
0,1960
1,1961
2,1962
3,1963
4,1964


In [15]:
# Testing cell
assert len(resultdf1) == 59

**Q2:** Using the table `indicators`, write a query to find all rows (you can select all fields for these rows) with no missing data for `gdp`.

In [16]:
query = """
SELECT * FROM indicators
WHERE gdp IS NOT NULL
"""
# YOUR CODE HERE
# raise NotImplementedError()

resultset2 = %sql $query
resultdf2 = resultset2.DataFrame()
resultdf2.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year,code,pop,gdp,life,cell,imports,exports
0,1960,AFG,9.00,0.54,32.3,0.0,NaN,49.9
1,1960,AUS,10.28,18.58,70.8,0.0,2524.06,2022.9
2,1960,AUT,7.05,6.59,68.6,0.0,1408.80,1118.9
3,1960,BDI,2.80,0.20,41.3,0.0,NaN,NaN
4,1960,BEL,9.15,11.66,69.7,0.0,NaN,NaN


In [17]:
# Testing cell
assert len(resultdf2) == 9660

**Q3:** Using the table `indicators`, write a query to find all rows with no missing data for any of the numeric fields.

In [18]:
query = """
SELECT * FROM indicators
WHERE gdp IS NOT NULL AND year IS NOT NULL AND pop IS NOT NULL AND life IS NOT NULL AND cell IS NOT NULL AND imports IS NOT NULL AND exports IS NOT NULL 
"""
# YOUR CODE HERE
# raise NotImplementedError()

resultset3 = %sql $query
resultdf3 = resultset3.DataFrame()
resultdf3.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year,code,pop,gdp,life,cell,imports,exports
0,1960,AUS,10.28,18.58,70.8,0.0,2524.06,2022.9
1,1960,AUT,7.05,6.59,68.6,0.0,1408.80,1118.9
2,1960,BEN,2.43,0.23,37.3,0.0,1.90,0.5
3,1960,BOL,3.66,0.37,42.1,0.0,71.10,67.9
4,1960,BRA,72.18,15.17,54.2,0.0,1586.75,1269.9


In [19]:
# Testing cell
assert len(resultdf3) == 6958

**Q4:** Using the table `indicators`, write a query to find all rows where exports is higher than imports. Select all fields for such rows.

In [20]:
query = """
SELECT * FROM indicators
WHERE exports > imports
"""
# YOUR CODE HERE
# raise NotImplementedError()

resultset4 = %sql $query
resultdf4 = resultset4.DataFrame()
resultdf4.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year,code,pop,gdp,life,cell,imports,exports
0,1960,BRN,0.08,NaN,62.5,0.0,1.30,3.30
1,1960,CIV,3.50,0.55,36.9,0.0,119.92,151.19
2,1960,CMR,5.18,0.62,41.5,0.0,84.10,96.90
3,1960,DEU,72.81,NaN,69.3,0.0,10013.00,11381.00
4,1960,DOM,3.29,0.67,51.8,0.0,95.92,179.70


In [21]:
# Testing cell
assert len(resultdf4) == 2492
assert resultdf4.shape == (2492,8)

**Q5:** Using the table `indicators`, write a query to find the minimum non-zero number for `cell` that appears.  Although you could use `MIN` (which we haven't seen yet), you should try to do this using `ORDER BY` instead, so that the first row in your result has that minimum value.

In [22]:
query = """
SELECT * FROM indicators
WHERE cell <> 0
ORDER BY cell ASC
LIMIT 1
"""
# YOUR CODE HERE
# raise NotImplementedError()

resultset5 = %sql $query
resultdf5 = resultset5.DataFrame()
resultdf5.head()

"""
CHECKPOINT 1:
SELECT count FROM indicators
WHERE year = 1961 AND sex = 'Female'
"""

 * sqlite:///../../dbfiles/book.db
Done.


"\nCHECKPOINT 1:\nSELECT count FROM indicators\nWHERE year = 1961 AND sex = 'Female'\n"

In [23]:
# Testing cell
assert resultdf5.loc[0,'cell'] == 0.01

> You've reached the first checkpoint in the lab.  Make sure to have it signed off by the instructor.
>
> Checkpoint 1: Convert the following `pandas` selection into a SQL query:
>     `topnames.loc[(1961, 'Female'), 'count']`

---

# Part D: Subqueries

We can use the result of a query as a building block in another query.  For example, we may want to know which countries have _ever_ had a GDP greater than `3000`:

In [24]:
query = """
SELECT DISTINCT code
FROM indicators
WHERE gdp > 3000
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


code
USA
JPN
DEU
CHN
GBR


Then, four just these five countries, we can look up their `2017` population and life expectancy values:

In [25]:
query = """
SELECT code, pop, life
FROM indicators
WHERE year = 2017 AND code IN ('USA', 'JPN', 'DEU', 'CHN', 'GBR')
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


code,pop,life
CHN,1386.4,76.4
DEU,82.66,81.0
GBR,66.06,81.2
JPN,126.79,84.1
USA,325.15,78.5


However, we don't always want to hard-code these country codes.  We can instead use the results of the first query as the `set-members` for the second query:

In [26]:
query = """
SELECT code, pop, life
FROM indicators
WHERE year = 2017 AND code IN (SELECT DISTINCT code FROM indicators WHERE gdp > 3000)
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


code,pop,life
CHN,1386.4,76.4
DEU,82.66,81.0
GBR,66.06,81.2
JPN,126.79,84.1
USA,325.15,78.5


As another example, we can query the most popular male baby name from 2016:

In [27]:
query = """
SELECT name
FROM topnames
WHERE year = 2016 AND sex = 'Male'
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


name
Noah


Then, we can use this result to find all years for which `Noah` was the most popular male baby name, and the associated counts:

In [28]:
query = """
SELECT year, count
FROM topnames
WHERE name = (SELECT name FROM topnames WHERE year = 2016 AND sex = 'Male')
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


year,count
2013,18257
2014,19305
2015,19635
2016,19117


Similarly, we can use a subquery as our table, rather than as part of a `WHERE` clause:

In [29]:
query = """
SELECT *
FROM countries
WHERE land < 100
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


code,country,region,income,land
BMU,Bermuda,North America,High income,54.0
GIB,Gibraltar,Europe & Central Asia,High income,10.0
MAC,"Macao SAR, China",East Asia & Pacific,High income,30.4
NRU,Nauru,East Asia & Pacific,Upper middle income,20.0
SMR,San Marino,Europe & Central Asia,High income,60.0
TUV,Tuvalu,East Asia & Pacific,Upper middle income,30.0


In [30]:
query = """
SELECT country AS Name, land AS Land
FROM (SELECT * FROM countries WHERE land < 100)
WHERE income = 'High income'
"""

%sql $query

 * sqlite:///../../dbfiles/book.db
Done.


Name,Land
Bermuda,54.0
Gibraltar,10.0
"Macao SAR, China",30.4
San Marino,60.0


**Q6:** Use a subquery to select the top ten entries in `indicators` with the highest population, then select all fields for the three from that group of ten that have the lowest GDP.

_Hint: You may find it helpful to first write the query just for the top ten entries in `indicators` with the highest population._

In [31]:
query = """
SELECT * FROM (SELECT * FROM indicators
ORDER BY pop DESC
LIMIT 10)
ORDER BY gdp ASC
LIMIT 3
"""
# YOUR CODE HERE
# raise NotImplementedError()

resultset6 = %sql $query
resultdf6 = resultset6.DataFrame()
resultdf6.head()

"""
CHECKPOINT 2:
gdp>3000 alone will produce more rows because we dont have another constraint of year = 2017. The query
wants to get all rows of many years
"""

 * sqlite:///../../dbfiles/book.db
Done.


'\nCHECKPOINT 2:\ngdp>3000 alone will produce more rows because we dont have another constraint of year = 2017. The query\nwants to get all rows of many years\n'

In [32]:
# Testing cell
assert len(resultdf6) == 3
assert resultdf6.loc[0,'gdp'] == 2652.55
assert resultdf6.loc[1,'pop'] == 1352.62

> You've reached the second (and final) checkpoint in the lab.  Make sure to have it signed off by the instructor.
>
> Checkpoint 2: Consider the following query from above:
> ```
>     SELECT code, pop, life
>     FROM indicators
>     WHERE year = 2017 AND code IN
>         (SELECT DISTINCT code FROM indicators WHERE gdp > 3000)
> ```
> How is this different from just looking for `gdp > 3000` within the `WHERE` clause?

---

---
## Part E

How much time (in minutes/hours) did you spend on this lab outside of class?

For about 15 minutes